<a href="https://colab.research.google.com/github/aryansonu2003/EE-Projects-/blob/main/Projects_health_monitoring_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Health Monitoring System (**S5**)**

In [3]:

%%writefile health_monitor.ino

#define BLYNK_TEMPLATE_ID "TMPL3zgEHUJZ1"
#define BLYNK_TEMPLATE_NAME "Health Monitoring System"
#include <Wire.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>
#include <Adafruit_MPU6050.h>
#include <Adafruit_Sensor.h>
#include <MAX30105.h>
#include <heartRate.h>
#include <ESP8266WiFi.h>
#include <BlynkSimpleEsp8266.h>

#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64
#define OLED_RESET -1
Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

MAX30105 particleSensor;
Adafruit_MPU6050 mpu;

char auth[] = "STORSTaYTvzNzX6VsS_iRRjgfdKlPsAy";  // Blynk Token
char ssid[] = "FACULTY";        // Wi-Fi SSID
char pass[] = "fac*2019#tasif";    // Wi-Fi Password

float ax, ay, az;

BlynkTimer timer;  // Timer for sending data to Blynk

// Buzzer and LED pins
const int buzzerPin = 5;
const int ledPin = 2;

void setup() {
  Serial.begin(115200);

  // Initialize OLED display
  if (!display.begin(SSD1306_SWITCHCAPVCC, 0x3C)) {
    Serial.println(F("SSD1306 allocation failed"));
    for (;;); // Loop forever if display initialization fails
  }
  display.clearDisplay();

  // Initialize MAX30105 particle sensor (heart rate and SpO2)
  if (!particleSensor.begin(Wire, I2C_SPEED_FAST)) {
    Serial.println(F("MAX30105 was not found. Please check wiring/power."));
    for (;;); // Loop forever if sensor initialization fails
  }
  particleSensor.setup();
  particleSensor.setPulseAmplitudeRed(0x0A);
  particleSensor.setPulseAmplitudeGreen(0);

  // Initialize MPU6050 (accelerometer)
  if (!mpu.begin()) {
    Serial.println("MPU6050 not found");
    while (1); // Loop forever if MPU6050 initialization fails
  }

  // Initialize Wi-Fi and Blynk
  Blynk.begin(auth, ssid, pass);

  // Initialize Buzzer and LED
  pinMode(buzzerPin, OUTPUT);
  pinMode(ledPin, OUTPUT);

  display.clearDisplay();
}

// Threshold values
const int heartRateMin = 60;
const int heartRateMax = 110;
const float SpO2Min = 95.0;
const float SpO2Max = 98.0;
const float tempMin = 36.0;
const float tempMax = 37.7;
const float accelXThreshold = 2.0;  // X-axis threshold for alert
const float accelZThreshold = 0.5;  // Z-axis threshold for fall detection

void loop() {
  if (WiFi.status() == WL_CONNECTED) {
    Blynk.run();  // Only run Blynk if WiFi is connected
  }

  long irValue = particleSensor.getIR();
  float temperature = 36.5; // Dummy value, replace with LM35 sensor data
  float SpO2 = 98.6;        // Dummy SpO2 value
  int heartRate = 72;       // Dummy heart rate value

  sensors_event_t a, g, temp;
  mpu.getEvent(&a, &g, &temp);
  ax = a.acceleration.x;
  ay = a.acceleration.y;
  az = a.acceleration.z;

  // Check thresholds and activate alarms
  checkThresholds(heartRate, SpO2, temperature, ax, az);

  // OLED Display Logic - cycle through the values every 5 seconds
  displayHeartRate(heartRate);
  delay(5000);

  displaySpO2(SpO2);
  delay(5000);

  displayTemperature(temperature);
  delay(5000);

  displayAccelerometer(ax, ay, az);
  delay(5000);
}

// Function to check thresholds and activate buzzer/LED if needed
void checkThresholds(int heartRate, float SpO2, float temperature, float ax, float az) {
  bool thresholdBreached = false;

  // Heart rate check
  if (heartRate < heartRateMin || heartRate > heartRateMax) {
    thresholdBreached = true;
    activateAlarm();
  }

  // SpO2 check
  if (SpO2 < SpO2Min || SpO2 > SpO2Max) {
    thresholdBreached = true;
    activateAlarm();
  }

  // Temperature check
  if (temperature < tempMin || temperature > tempMax) {
    thresholdBreached = true;
    activateAlarm();
  }

  // Accelerometer checks
  if (ax > accelXThreshold) {
    // Only LED for X-axis breach
    activateLED();
  }
  if (az > accelZThreshold) {
    thresholdBreached = true;
    activateAlarm();  // Fall detected
  }

  // Deactivate alarm if no thresholds breached
  if (!thresholdBreached) {
    deactivateAlarm();
  }
}

// Function to activate the alarm (buzzer and LED)
void activateAlarm() {
  digitalWrite(buzzerPin, HIGH);
  digitalWrite(ledPin, HIGH);
  delay(50);
  digitalWrite(buzzerPin, LOW);
  digitalWrite(ledPin, LOW);
  delay(50);
}

// Function to activate LED only (for X-axis threshold breach)
void activateLED() {
  digitalWrite(ledPin, HIGH);
  delay(1000);
  digitalWrite(ledPin, LOW);
  delay(1000);
}

// Function to deactivate the alarm
void deactivateAlarm() {
  digitalWrite(buzzerPin, LOW);
  digitalWrite(ledPin, LOW);
}

// OLED display functions (simplified)
void displayHeartRate(int heartRate) {
  display.clearDisplay();
  display.setTextSize(1);
  display.setTextColor(SSD1306_WHITE);
  display.setCursor(0, 0);
  display.print("Heart Rate: ");
  display.print(heartRate);
  display.print(" BPM");
  display.display();
}

void displaySpO2(float SpO2) {
  display.clearDisplay();
  display.setTextSize(1);
  display.setTextColor(SSD1306_WHITE);
  display.setCursor(0, 0);
  display.print("SpO2: ");
  display.print(SpO2);
  display.print(" %");
  display.display();
}

void displayTemperature(float temperature) {
  display.clearDisplay();
  display.setTextSize(1);
  display.setTextColor(SSD1306_WHITE);
  display.setCursor(0, 0);
  display.print("Temperature: ");
  display.print(temperature);
  display.print(" C");
  display.display();
}

void displayAccelerometer(float ax, float ay, float az) {
  display.clearDisplay();
  display.setTextSize(1);
  display.setTextColor(SSD1306_WHITE);
  display.setCursor(0, 0);
  display.print("Accel X: ");
  display.print(ax);
  display.setCursor(0, 10);
  display.print("Accel Y: ");
  display.print(ay);
  display.setCursor(0, 20);
  display.print("Accel Z: ");
  display.print(az);
  display.display();
}

Writing health_monitor.ino
